###### Team IPPS 

The API information can be accessed through this [link](https://data.cms.gov/Medicare-Inpatient/Inpatient-Prospective-Payment-System-IPPS-Provider/97k6-zzx3)

Objectives (This will expand from just data frame formation to include cleanup by Jessie):
1. Execute a successful pull of all of the data using the API provided into JSON
2. Transform the JSON format into a data frame


### Step 1: Installing sodapy, a python package, in jupyter

In [1]:
# Installing the package sodapy, important for retrieval using the API's detailed method
import sys
!{sys.executable} -m pip install sodapy

  Running setup.py bdist_wheel for future: started
  Running setup.py bdist_wheel for future: finished with status 'done'
  Stored in directory: C:\Users\Luffy4G\AppData\Local\pip\Cache\wheels\bf\c9\a3\c538d90ef17cf7823fa51fc701a7a7a910a80f6a405bf15b1a
Successfully built future


You are using pip version 10.0.1, however version 18.1 is available.
You should consider upgrading via the 'python -m pip install --upgrade pip' command.


### Step 2: Retrieving the dataset

In [13]:
# The API link
link = "https://data.cms.gov/resource/ehrv-m9r6.json"

# The app token
key = "oBbcgRhXZS4dqtTJVyz6zQujv"

# Code snippet for data retrieval using python, as provided by the API information page
import pandas as pd
from sodapy import Socrata

# Unauthenticated client only works with public data sets. Note 'None'
# in place of application token, and no username or password:
client = Socrata("data.cms.gov", key)

# Example authenticated client (needed for non-public datasets):
# client = Socrata(data.cms.gov,
#                  MyAppToken,
#                  userame="user@example.com",
#                  password="AFakePassword")

# First 2000 results, returned as JSON from API / converted to Python list of
# dictionaries by sodapy.
results = client.get("ehrv-m9r6", limit=163065)

# Convert to pandas DataFrame
results_df = pd.DataFrame.from_records(results)

# Checking the head of the data frame
results_df.tail(100)

,average_covered_charges,average_medicare_payments,average_medicare_payments_2,drg_definition,hospital_referral_region_description,provider_city,provider_id,provider_name,provider_state,provider_street_address,provider_zip_code,total_discharges
162965,32874.08,4108.68,3381.84,948 - SIGNS & SYMPTOMS W/O MCC,VA - Richmond,RICHMOND,490118,HENRICO DOCTORS' HOSPITAL,VA,1602 SKIPWITH ROAD,23229,25
162966,12810.78,3732.72,3126.78,948 - SIGNS & SYMPTOMS W/O MCC,VA - Norfolk,CHESAPEAKE,490120,CHESAPEAKE REGIONAL MEDICAL CENTER,VA,736 BATTLEFIELD BLVD NORTH,23320,33
162967,14454.45,3874,3139.8,948 - SIGNS & SYMPTOMS W/O MCC,VA - Roanoke,LOW MOOR,490126,LEWISGALE HOSPITAL ALLEGHANY,VA,ONE ARH LANE - PO BOX 7,24457,20
162968,10355.22,3473.61,2848.27,948 - SIGNS & SYMPTOMS W/O MCC,VA - Newport News,GLOUCESTER,490130,RIVERSIDE WALTER REED HOSPITAL,VA,7519 HOSPITAL ROAD,23061,18
162969,18125.78,4430.21,3363.78,948 - SIGNS & SYMPTOMS W/O MCC,WA - Seattle,SEATTLE,500001,NORTHWEST HOSPITAL,WA,1550 NORTH 115TH STREET,98133,14
162970,24134.04,4605.34,3871.26,948 - SIGNS & SYMPTOMS W/O MCC,WA - Everett,MOUNT VERNON,500003,SKAGIT VALLEY HOSPITAL,WA,1415 KINCAID STREET,98274,23
162971,10787.54,5902.41,4440,948 - SIGNS & SYMPTOMS W/O MCC,WA - Seattle,SEATTLE,500005,VIRGINIA MASON MEDICAL CENTER,WA,1100 NINTH AVENUE (PO BOX 900),98111,24
162972,18125.65,8771.80,6833.42,948 - SIGNS & SYMPTOMS W/O MCC,WA - Seattle,SEATTLE,500008,UNIVERSITY OF WASHINGTON MEDICAL CTR,WA,1959 NE PACIFIC ST,98195,26
162973,14726.14,4554.57,3629.57,948 - SIGNS & SYMPTOMS W/O MCC,WA - Yakima,YAKIMA,500012,YAKIMA REGIONAL MEDICAL AND CARDIAC CENTER,WA,110 SOUTH NINTH AVE,98902,21
162974,18773.61,4618.16,3811.27,948 - SIGNS & SYMPTOMS W/O MCC,WA - Everett,EVERETT,500014,PROVIDENCE REGIONAL MEDICAL CENTER EVERETT,WA,1321 COLBY AVENUE,98201,18


In [5]:
resultsGB = results_df.groupby("provider_state")

In [6]:
resultsGB

In [11]:
resultsGB_data = pd.DataFrame(resultsGB["drg_definition"].nunique())

In [12]:
resultsGB_data

,drg_definition
provider_state,
AK,78
AL,100
AR,100
AZ,100
CA,100
CO,100
CT,100
DC,100
DE,100


In [14]:
resultGB = results_df.groupby(["provider_state","drg_definition"]).size().reset_index(name='counts')
resultGB = pd.DataFrame(resultGB)
resultGB.sort_values(["drg_definition", "counts"], ascending = [True, False])

,provider_state,drg_definition,counts
878,FL,039 - EXTRACRANIAL PROCEDURES W/O CC/MCC,81
4246,TX,039 - EXTRACRANIAL PROCEDURES W/O CC/MCC,78
378,CA,039 - EXTRACRANIAL PROCEDURES W/O CC/MCC,67
1361,IL,039 - EXTRACRANIAL PROCEDURES W/O CC/MCC,52
3447,OH,039 - EXTRACRANIAL PROCEDURES W/O CC/MCC,48
3347,NY,039 - EXTRACRANIAL PROCEDURES W/O CC/MCC,46
3746,PA,039 - EXTRACRANIAL PROCEDURES W/O CC/MCC,44
2160,MI,039 - EXTRACRANIAL PROCEDURES W/O CC/MCC,40
1461,IN,039 - EXTRACRANIAL PROCEDURES W/O CC/MCC,37
4146,TN,039 - EXTRACRANIAL PROCEDURES W/O CC/MCC,32
